# Overview
Compute the committor using FEM methods for the triple well problem.  This code requires
* FEniCSx (checked with versino 0.7.3) with PETSc and MPI
* gmsh
* NumPy/SciPy/Matplotlib

In [ ]:
import gmsh
import numpy as np
from matplotlib import pyplot as plt
from dolfinx.io import gmshio
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
from dolfinx import fem
from dolfinx.plot import vtk_mesh
import pyvista
from dolfinx import default_scalar_type
import ufl
import dolfinx
dolfinx.__version__
from petsc4py import PETSc
import scipy.sparse
import scipy.sparse.linalg
from dolfinx import geometry
import scipy.io

# Setup Potential and Sets

In [ ]:
def three_well(x,y):
    return ((3.0*np.exp(-x**2-(y-1.0/3.0)**2) - 3.0*np.exp(-x**2-(y-5.0/3.0)**2) 
             - 5.0*np.exp(-(x-1)**2-y**2)-5.0*np.exp(-(x+1)**2-y**2)+0.2*x**4+0.2*(y-1.0/3.0)**4))


In [ ]:
xA = np.array([-1, 0])
xB = np.array([1, 0])
rA = 0.5
rB = 0.5

In [ ]:
xx = np.linspace(-2, 2,num=101)
yy = np.linspace(-1.5,2.5,num=101)
XX, YY = np.meshgrid(xx,yy)
Vvals = three_well(XX,YY)
plt.figure()
ax = plt.gca()
plt.contourf(XX,YY,Vvals,np.linspace(-4,8,num=51),cmap=plt.cm.RdBu_r, alpha = 0.75)
circle_left = plt.Circle(xA, rA, color="r", fill=False,lw=2,ls="--")
circle_right = plt.Circle(xB, rB, color="g", fill=False,lw=2,ls="--")
ax.add_patch(circle_left)
ax.add_patch(circle_right)

cbar = plt.colorbar()
cbar.set_label('$V(x,y)$')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.show()

# Construct Mesh

In [ ]:
gmsh.initialize()

In [ ]:
box = gmsh.model.occ.addRectangle(-3,-2,0,6,5)
hole_left = gmsh.model.occ.addDisk(xA[0],xA[1],0,rA, rA)
hole_right = gmsh.model.occ.addDisk(xB[0],xB[1],0,rB, rB)

In [ ]:
gmsh.model.occ.cut([(2, box)], [(2, hole_left)])
gmsh.model.occ.cut([(2, box)], [(2, hole_right)])

In [ ]:
gmsh.model.occ.synchronize()

In [ ]:
volumes = gmsh.model.getEntities(dim=2)
gdim = 2
gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], 1)
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.05)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.1)
gmsh.model.mesh.generate(2)

## Inspect Mesh

In [ ]:
gmsh_model_rank = 0
mesh_comm = MPI.COMM_WORLD
domain, cell_markers, facet_markers = gmshio.model_to_mesh(gmsh.model, 
                                                           mesh_comm, 
                                                           gmsh_model_rank, gdim=gdim)

In [ ]:
gmsh.clear()

In [ ]:

V = fem.FunctionSpace(domain, ("Lagrange", 1))
topology, cell_types, x = vtk_mesh(V)

In [ ]:
grid = pyvista.UnstructuredGrid(topology, cell_types, x)
plotter = pyvista.Plotter()
plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()
plotter.show()

# Set up FEM Problem and Solve

In [ ]:
β  = 6.67

def μ(x):
    return np.exp(-β*three_well(x[0],x[1]))

## Set up Boundary Conditions

In [ ]:
def on_boundary_left(x):
    return np.isclose((x[0]-xA[0])**2 + (x[1]-xA[1])**2, rA**2)

def on_boundary_right(x):
    return np.isclose((x[0]-xB[0])**2 + (x[1]-xB[1])**2, rB**2)

boundary_left_dofs = fem.locate_dofs_geometrical(V, on_boundary_left)
boundary_right_dofs = fem.locate_dofs_geometrical(V, on_boundary_right)

bc_left = fem.dirichletbc(value=default_scalar_type(0.), dofs=boundary_left_dofs, V=V)
bc_right = fem.dirichletbc(value=default_scalar_type(1.), dofs=boundary_right_dofs, V=V)

## Set up Weak Form and Solve

In [ ]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

μ_fem = fem.Function(V)
μ_fem.interpolate(μ)

In [ ]:
f = fem.Constant(domain, default_scalar_type(0.0))
a = ufl.inner( ufl.grad(u), ufl.grad(v)) *μ_fem* ufl.dx
L = f * v * ufl.dx

In [ ]:
solver_opts={"ksp_type": "gmres", "pc_type": "gamg","ksp_monitor": "", "ksp_rtol":1e-8}

In [ ]:
problem = LinearProblem(a, L, bcs=[bc_left, bc_right],petsc_options=solver_opts)


In [ ]:
uh = problem.solve()

Check solver performance

In [ ]:
solver = problem.solver

solver.getIterationNumber()

In [ ]:
solver.getConvergedReason()

Check condition number, if desired

In [ ]:
A_csr = problem.A.getValuesCSR()
A_sparse = scipy.sparse.csr_array((A_csr[2], A_csr[1], A_csr[0]))
A_sparse.shape

In [ ]:
σs = scipy.sparse.linalg.svds(A_sparse, k=507, return_singular_vectors=False)

In [ ]:
σs[-1]/σs[0]

## Check Solution

In [ ]:
pyvista.start_xvfb()
u_topology, u_cell_types, u_geometry = vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)
u_grid.point_data["u"] = uh.x.array.real
u_grid.set_active_scalars("u")
u_plotter = pyvista.Plotter()
u_plotter.add_mesh(u_grid, show_edges=False)
u_plotter.view_xy()
u_plotter.show()

In [ ]:
warped = u_grid.warp_by_scalar()
plotter = pyvista.Plotter()
plotter.add_mesh(warped, show_edges=True, show_scalar_bar=True)
plotter.show()


# Dump to disk

## On Regular Mesh

In [ ]:
tol = 0.  # Avoid hitting the outside of the domain

x = np.linspace(-2 + tol, 2 - tol, 201)
y = np.linspace(-1.5 + tol, 2.5 - tol, 401)

points = np.zeros((3, 0))
points_notinAB = np.zeros((3, 0))
points_inAB = np.zeros((3, 0))
def notinAB(X):
    return ((X[0] - xA[0])**2 + (X[1] - xA[1])**2 > rA**2 ) and ((X[0] - xB[0])**2 + (X[1] - xB[1])**2 > rB**2 )

inABidx =np.array([],dtype=int)
notinABidx =np.array([],dtype=int)
i = 0

for y_ in y:
    for x_ in x:
        if notinAB([x_, y_]):
            notinABidx = np.append(notinABidx, i)
            points_notinAB = np.concatenate((points_notinAB, np.array([[x_,y_,0.]]).T),axis=1)
        else:
            inABidx = np.append(inABidx, i)
            points_inAB = np.concatenate((points_inAB, np.array([[x_,y_,0.]]).T),axis=1)
        i+=1
        points = np.concatenate((points, np.array([[x_,y_,0.]]).T),axis=1)

u_values = []

In [ ]:
print(len(notinABidx))
print(len(inABidx))
print(len(notinABidx)+len(inABidx))

In [ ]:
plt.scatter(points_notinAB[0], points_notinAB[1])
plt.scatter(points_inAB[0], points_inAB[1])
plt.show()

In [ ]:
bb_tree = geometry.bb_tree(domain, domain.topology.dim)

In [ ]:
cells = []
points_on_proc = []
# Find cells whose bounding-box collide with the the points
cell_candidates = geometry.compute_collisions_points(bb_tree, points_notinAB.T)
# Choose one of the cells that contains the point
colliding_cells = geometry.compute_colliding_cells(domain, cell_candidates, points_notinAB.T)
for i, point in enumerate(points_notinAB.T):
    if len(colliding_cells.links(i)) > 0:
        points_on_proc.append(point)
        cells.append(colliding_cells.links(i)[0])

In [ ]:
len(cells)

In [ ]:
points_on_proc = np.array(points_on_proc, dtype=np.float64)
u_values = uh.eval(points_on_proc, cells)

In [ ]:
len(u_values)
u_values.size

In [ ]:
print(points_notinAB[0].shape)
print(points_notinAB[1].shape)
print(u_values.flatten().shape)

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': '3d'})
ax.plot_trisurf(points_notinAB[0], points_notinAB[1], u_values.flatten())
plt.show()



In [ ]:
u_pad = np.zeros(x.shape[0] * y.shape[0])
u_pad[notinABidx] = u_values.flatten()
u_pad[inABidx] = np.NaN

In [ ]:
len(u_pad)

In [ ]:
matlab_data = {"x":x, "y":y, "u":u_pad.flatten(), "beta":β}
scipy.io.savemat("committor_beta{beta}_n{n}.mat".format(beta=β, n=len(u_pad)), matlab_data)

In [ ]:
filename = "committor_beta{beta}_n{n}.npz".format(beta = β, n = u_values.size)
print(filename)
np.savez(filename, u = u_values.flatten(), xx=x, yy=y,
         x = points[0], y = points[1], 
         x_inAB = points_inAB[0], y_inAB = points_inAB[1],
         x_notinAB = points_notinAB[0], y_notinAB = points_notinAB[1],
         notinABidx = notinABidx, inABidx = inABidx)

## Dump Nodal Data

In [ ]:
uh.function_space.tabulate_dof_coordinates().shape

In [ ]:
uh.vector[:].size

In [ ]:
matlab_data = {"xyz": uh.function_space.tabulate_dof_coordinates(), "u": uh.vector[:], "beta":β}

In [ ]:
scipy.io.savemat("committor_beta{beta}.mat".format(beta=β), matlab_data)